Monte Carlo Policy Optimization,
the algorithm given on page 101 of Sutton & Barton
for the game of gridworld. 

In [ ]:
import numpy as np
import random
from collections import defaultdict

WORLD_SIZE = 4
# left, up, right, down
ACTIONS = [(0, -1),
           (-1, 0),
           (0, 1),
           (1, 0)]
ACTION_PROB = 0.25
ACTIONS_FIGS=[ '←', '↑', '→', '↓']

#check if the spot is final destination
def is_terminal(state):
    x, y = state
    return (x == 0 and y == 0) or (x == WORLD_SIZE - 1 and y == WORLD_SIZE - 1)

#function to move
def step(state, action):
    if is_terminal(state):
        return state, 0

    next_state = (np.array(state) + action).tolist()
    x, y = next_state

    if x < 0 or x >= WORLD_SIZE or y < 0 or y >= WORLD_SIZE:
        next_state = state

    reward = -1
    return next_state, reward 

In [ ]:
#our initial environment
env1=([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [ ]:
#one episode generator. Takes into account epsilon greedy factor.
def generate_episode(Q5, eps):
    i=random.randint(0,3)
    j=random.randint(0,3)
    trajectory = []
    while True:
       SS = (i,j)
       if np.random.rand() > eps:
         temp1={}
         for a in ACTIONS:
           temp1[(SS,(a))] = Q5[(SS,(a))]
         max_key = max(temp1, key=temp1.get)  
         action=max_key[1]
       else:
          action = ACTIONS[random.randint(0,3)]
     
       (next_i, next_j), reward = step([i, j], action)
       trajectory.append(((i, j), reward, action))
       i= next_i
       j = next_j
       if  (i == 0 and j == 0) or (i == WORLD_SIZE - 1 and j == WORLD_SIZE - 1):  break
    return trajectory, len(trajectory)-1

In [ ]:
#main function. Generates Q(State-Action,Value) and pi(State-Action, probability)
def on_policy_MC_control11(ep, gamma, eps):
    """ ep - number of episodes to run
        gamma - discount factor
        eps - epsilon-greedy parameter
    """ 
    pi = defaultdict(lambda: 1/4)
    Returns = defaultdict(list)  # dict of lists
    #by Kamali 
    for _ in range(ep):
      traj, T = generate_episode(Q5, eps)
      G = 0
      for t in range(T-1,-1,-1):
            St, _, At = traj[t]      # (st, rew, act)
            _, Rt_1 , _ = traj[t+1]
            
            G = gamma * G + Rt_1
            
            if not (St, At) in [(traj[i][0], traj[i][2]) for i in range(0, t)]:
                Returns[(St, At)].append(G)
                Q5[(St, At)] = (np.average(Returns[(St, At)]))
                temp2={}
                for a in ACTIONS:
                  temp2[(St,(a))] = Q5[(St,(a))]
                max_key = max(temp2, key=temp2.get)  
                A_star=max_key[1]
                #A_star = argmax_rand([Q[(St,a)] for a in ACTIONS])  
                for a in ACTIONS:
                    if a == A_star:   pi[(St,a)] = 1 - eps + eps/4
                    else:             pi[(St,a)] = eps/4
                        
    return Q5, pi

In [ ]:
#Q table of the initital stage
Q5 = {((0, 3), (-1, 0)): 0,  ((0, 3), (0, 1)): 0, ((0, 3), (1, 0)): 0, ((0, 3), (0, -1)): 0, 
     ((0, 2), (-1, 0)): 0,  ((0, 2), (0, 1)): 0, ((0, 2), (1, 0)): 0, ((0, 2), (0, -1)): 0, 
     ((0, 1), (-1, 0)): 0,  ((0, 1), (0, 1)): 0, ((0, 1), (1, 0)): 0, ((0, 1), (0, -1)): 0, 
     ((0, 0), (-1, 0)): 0,  ((0, 0), (0, 1)): 0, ((0, 0), (1, 0)): 0, ((0, 0), (0, -1)): 0, 
     ((1, 3), (-1, 0)): 0,  ((1, 3), (0, 1)): 0, ((1, 3), (1, 0)): 0, ((1, 3), (0, -1)): 0, 
     ((1, 2), (-1, 0)): 0,  ((1, 2), (0, 1)): 0, ((1, 2), (1, 0)): 0, ((1, 2), (0, -1)): 0, 
     ((1, 1), (-1, 0)): 0,  ((1, 1), (0, 1)): 0, ((1, 1), (1, 0)): 0, ((1, 1), (0, -1)): 0, 
     ((1, 0), (-1, 0)): 0,  ((1, 0), (0, 1)): 0, ((1, 0), (1, 0)): 0, ((1, 0), (0, -1)): 0,
     ((2, 3), (-1, 0)): 0,  ((2, 3), (0, 1)): 0, ((2, 3), (1, 0)): 0, ((2, 3), (0, -1)): 0, 
     ((2, 2), (-1, 0)): 0,  ((2, 2), (0, 1)): 0, ((2, 2), (1, 0)): 0, ((2, 2), (0, -1)): 0, 
     ((2, 1), (-1, 0)): 0,  ((2, 1), (0, 1)): 0, ((2, 1), (1, 0)): 0, ((2, 1), (0, -1)): 0, 
     ((2, 0), (-1, 0)): 0,  ((2, 0), (0, 1)): 0, ((2, 0), (1, 0)): 0, ((2, 0), (0, -1)): 0, 
     ((3, 3), (-1, 0)): 0,  ((3, 3), (0, 1)): 0, ((3, 3), (1, 0)): 0, ((3, 3), (0, -1)): 0, 
     ((3, 2), (-1, 0)): 0,  ((3, 2), (0, 1)): 0, ((3, 2), (1, 0)): 0, ((3, 2), (0, -1)): 0, 
     ((3, 1), (-1, 0)): 0,  ((3, 1), (0, 1)): 0, ((3, 1), (1, 0)): 0, ((3, 1), (0, -1)): 0, 
     ((3, 0), (-1, 0)): 0,  ((3, 0), (0, 1)): 0, ((3, 0), (1, 0)): 0, ((3, 0), (0, -1)): 0,  
     }

In [ ]:
#run the function
Q, pi = on_policy_MC_control11(ep=300000, gamma=1, eps=0.1)

In [ ]:
#get unique states in the environment
address1=[]
keys=Q5.keys()
for i in keys:
  if i[0] != (0, 0):
    if i[0] != (3,3):
      address1.append(i[0])
address=set(address1)  

In [ ]:
#our Q(State-Action, Value)
Q

{((0, 0), (-1, 0)): 0,
 ((0, 0), (0, -1)): 0,
 ((0, 0), (0, 1)): 0,
 ((0, 0), (1, 0)): 0,
 ((0, 1), (-1, 0)): -1.1279782164737917,
 ((0, 1), (0, -1)): 0,
 ((0, 1), (0, 1)): -2.2841379310344827,
 ((0, 1), (1, 0)): -2.2612966601178783,
 ((0, 2), (-1, 0)): -2.3199598796389167,
 ((0, 2), (0, -1)): -1.140703116689714,
 ((0, 2), (0, 1)): -3.3379446640316206,
 ((0, 2), (1, 0)): -3.28041825095057,
 ((0, 3), (-1, 0)): -3.3611650485436892,
 ((0, 3), (0, -1)): -2.279051670471052,
 ((0, 3), (0, 1)): -3.353281853281853,
 ((0, 3), (1, 0)): -2.302325581395349,
 ((1, 0), (-1, 0)): 0,
 ((1, 0), (0, -1)): -1.1317414573898723,
 ((1, 0), (0, 1)): -2.2170361726954493,
 ((1, 0), (1, 0)): -2.248768472906404,
 ((1, 1), (-1, 0)): -1.1587591240875912,
 ((1, 1), (0, -1)): -1.1417876241405653,
 ((1, 1), (0, 1)): -3.2432179607109446,
 ((1, 1), (1, 0)): -3.254794520547945,
 ((1, 2), (-1, 0)): -2.2640625,
 ((1, 2), (0, -1)): -2.247014003294893,
 ((1, 2), (0, 1)): -2.26027397260274,
 ((1, 2), (1, 0)): -2.265700483091

In [ ]:
#our pi (State-Action, probability). This is not necessary as we already have Q but nice to have
pi

defaultdict(<function __main__.on_policy_MC_control11.<locals>.<lambda>>,
            {((0, 1), (-1, 0)): 0.025,
             ((0, 1), (0, -1)): 0.925,
             ((0, 1), (0, 1)): 0.025,
             ((0, 1), (1, 0)): 0.025,
             ((0, 2), (-1, 0)): 0.025,
             ((0, 2), (0, -1)): 0.925,
             ((0, 2), (0, 1)): 0.025,
             ((0, 2), (1, 0)): 0.025,
             ((0, 3), (-1, 0)): 0.025,
             ((0, 3), (0, -1)): 0.925,
             ((0, 3), (0, 1)): 0.025,
             ((0, 3), (1, 0)): 0.025,
             ((1, 0), (-1, 0)): 0.925,
             ((1, 0), (0, -1)): 0.025,
             ((1, 0), (0, 1)): 0.025,
             ((1, 0), (1, 0)): 0.025,
             ((1, 1), (-1, 0)): 0.025,
             ((1, 1), (0, -1)): 0.925,
             ((1, 1), (0, 1)): 0.025,
             ((1, 1), (1, 0)): 0.025,
             ((1, 2), (-1, 0)): 0.025,
             ((1, 2), (0, -1)): 0.925,
             ((1, 2), (0, 1)): 0.025,
             ((1, 2), (1, 0)): 0.025,
  

In [ ]:
#optimal pi with State and the best action to take
optimal_pi={}
for i in address:
  temp={}
  for a in ACTIONS:
    temp[((i),(a))] = pi[((i),(a))]
  max_key = max(temp, key=temp.get)  
  optimal_pi[max_key[0]]=max_key[1]
optimal_pi

{(0, 1): (0, -1),
 (0, 2): (0, -1),
 (0, 3): (0, -1),
 (1, 0): (-1, 0),
 (1, 1): (0, -1),
 (1, 2): (0, -1),
 (1, 3): (1, 0),
 (2, 0): (-1, 0),
 (2, 1): (0, 1),
 (2, 2): (1, 0),
 (2, 3): (1, 0),
 (3, 0): (-1, 0),
 (3, 1): (0, 1),
 (3, 2): (0, 1)}

In [ ]:
#replace action with arrows for easy reading
optimal_pi_dir={}
for spot, dir in optimal_pi.items():
  if dir == (0, -1):
    optimal_pi_dir[spot]='←'
  elif dir == (0, 1):
    optimal_pi_dir[spot]='→'
  elif dir == (1, 0):
    optimal_pi_dir[spot]='↓'
  elif dir == (-1, 0):
    optimal_pi_dir[spot]='↑'  
optimal_pi_dir

{(0, 1): '←',
 (0, 2): '←',
 (0, 3): '←',
 (1, 0): '↑',
 (1, 1): '←',
 (1, 2): '←',
 (1, 3): '↓',
 (2, 0): '↑',
 (2, 1): '→',
 (2, 2): '↓',
 (2, 3): '↓',
 (3, 0): '↑',
 (3, 1): '→',
 (3, 2): '→'}

In [ ]:
#our final environment with the solution(best move) in each state as per our function
for i in optimal_pi_dir.keys():
  env1[i[0]][i[1]]=optimal_pi_dir[i]
env1

[[0, '←', '←', '←'],
 ['↑', '←', '←', '↓'],
 ['↑', '→', '↓', '↓'],
 ['↑', '→', '→', 0]]